# 读取IOB格式和分块语料库

In [1]:
import nltk
text ='''he PRP B-VP
accepted VBD B-VP
the DT B-NP
position NN I-NP
of IN B-PP
vice NN B-NP
chairman NN I-NP
of IN B-PP
Carlyle NNP B-NP
Group NNP I-NP
a DT B-NP
merchant NN I-NP
banking NN I-NP
concern NN I-NP
'''
nltk.chunk.conllstr2tree(text, chunk_types=('NP')).draw()

In [2]:
from nltk.corpus import conll2000
print (conll2000.chunked_sents('train.txt',chunk_types=['NP'])[99])

(S
  Over/IN
  (NP a/DT cup/NN)
  of/IN
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  told/VBD
  (NP his/PRP$ story/NN)
  ./.)


# 简单评估和基准

我们开始为琐碎的不创建任何 块的块分析器 cp 建立一个基准

现在让我们尝试一个初级 的正则表达式分块器，查找以名词短语标记的特征字母(如 CD、DT 和 JJ)开头的标记

In [3]:
cp = nltk.RegexpParser("")
test_sents = conll2000.chunked_sents('test.txt',chunk_types=['NP'])
print (cp.evaluate(test_sents))
grammar = "NP: {<[CDJNP].*>+}"
cp = nltk.RegexpParser(grammar)
print (cp.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  43.4%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%
ChunkParse score:
    IOB Accuracy:  87.7%%
    Precision:     70.6%%
    Recall:        67.8%%
    F-Measure:     69.2%%


In [6]:
#使用n-gram标注器对名词短语分块
class ngramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        '''类初始化 
        chunk.tree2conlltags 返回包含（word、tag、iob标记）的三元组列表。将树转换为Conll IOB标记格式
        UnigramTagger(train_data) 将unigram标注器训练成一元分块器
        BigramTagger(train_data) 将Bigram标注器训练成二元分块器
        '''
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.tag.BigramTagger(train_data)
    
    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]#获取词性标记数列
        tagged_pos_tags = self.tagger.tag(pos_tags)#对数列进行分块
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]#获取块数据
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]#组合打包词、词性、块数据
        return nltk.chunk.conlltags2tree(conlltags)#返回树结构

test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
train_sents = conll2000.chunked_sents('train.txt', chunk_types=['NP'])
ngram_chunker = ngramChunker(train_sents)
print(ngram_chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  93.3%%
    Precision:     82.3%%
    Recall:        86.8%%
    F-Measure:     84.5%%


# 训练基于分类器的分块器